# Near Duplicate Methods Comparison

The purpose of this notebook is to outline, compare and evaluate the following methods for near-duplicate detection:
1. Brute-Force
2. Length-based filtering
3. Prefix-based filtering
4. Minhash-LSH via
    1. 9-char shingles
    2. unigram shingles
    3. bigram shingles
    4. trigram shingles
    5. filtered trigram shingles


The methods are based on chapter 3 of ["Mining of Massive Datasets"](http://www.mmds.org/). 


For the purposes of this notebook, we will define that two documents A and B are near-duplicates if the Jaccard similarity of their trigrams is at least 0.8.


In [2]:
J = 0.8

def are_near_duplicates(A:set, B:set)-> bool:
    return len(A.intersection(B)) / float(len(A.union(B))) > J

# Data Preparation

The research will be done on a proprietary dataset of news articles. We shall rely on space for the tokenization.

In [3]:
import spacy
import pandas as pd
import fastprogress

In [30]:
nlp = spacy.load('en')

nlp.remove_pipe('tagger')
nlp.remove_pipe('parser')
nlp.remove_pipe('ner')

('ner', <spacy.pipeline.EntityRecognizer at 0x7fae9df24bf8>)

In [20]:
data = pd.read_excel('centrica.xlsx', dtype={})
data['text'] = data.Title.str.cat(data.Body.fillna(""), sep=" endoftitle ")

In [31]:
data['docs'] = list(fastprogress.progress_bar(nlp.pipe(data['text']),total=len(data) ))

In [32]:
def trigrams(doc: spacy.tokens.Doc):
    r = set()
    for i in range(len(doc) - 2):
        r.add(" ".join([t.lower_ for t in doc[i:i+3]]))
    return r

So, the trigrams for the first document would be:


In [33]:
trigrams(data.docs.iloc[0])

{'\n\n centrica noted',
 '\n\n centrica said',
 '\n\n he succeeds',
 '\n\n mr berry',
 '\n\n mr haythornthwaite',
 '\n\n “ charles',
 ', and a',
 ', and retail',
 ', and telecoms',
 ', commodity trading',
 ', electricity generator',
 ', energy retailing',
 ', in previous',
 ', minerals ,',
 ', mr berry',
 ', on an',
 ', stand him',
 ', was appointed',
 ', who announced',
 ', who is',
 '- based engineering',
 '- executive director',
 '. \n\n centrica',
 '. \n\n he',
 '. \n\n mr',
 '. \n\n “',
 '. charles also',
 '. i have',
 '. mr berry',
 '. ” \n\n',
 '12 months after',
 '20 years ,',
 '21 . \n\n',
 '445,000 . \n\n',
 ': “ i',
 ': “ it',
 'a long track',
 'a non -',
 'a privilege to',
 'a recruitment process',
 'about six years',
 'across industrial ,',
 'admired centrica for',
 'after about six',
 'also chairman of',
 'also has extensive',
 'am delighted to',
 'an annual fee',
 'an excellent position',
 'and a long',
 'and engineering knowledge',
 'and executive team',
 'and follow ri

In [34]:
# calculate trigrams for the entire dataset
data['trigrams'] = data.docs.apply(trigrams)

Let's check a pair of duplicates, documents 21 and 22:

In [38]:
data.text[:50]

0     Scots corporate veteran is new Centrica chairm...
1     FTSE 100 surges higher as global markets recov...
2     Centrica: Owner of British Gas selects new cha...
3     Scots corporate veteran is new Centrica chairm...
4     Centrica: Chairman chosen endoftitle Chairman ...
5     Centrica forms strategic partnership with Micr...
6     MICROSOFT CORPORATION (MSFT) endoftitle Micros...
7     New gas storage expected to be ‘very small sca...
8     Departing DWP digital chief Prakash to take tr...
9     Microsoft: Follow the Day 2 live blog from Fut...
10    Are UK companies at risk of falling behind due...
11    Follow the Day 2 live blog from Future Decoded...
12    Microsoft deploys new software, services platf...
13    Centrica forms strategic partnership with Micr...
14    Centrica: Powering our field operations with c...
15    Microsoft announces new strategic partnerships...
16    Mayank Prakash joins Centrica as Ash Jokhoo mo...
17    PM meets European Round Table of Industria

In [58]:
idx, idy = (386, 605)

display("-------\First document\n---------")
display(data.text[idx][:1000])
display("-------\nSecond document\n---------")
display(data.text[idy][:1000])
display(f"Are near duplicates: {are_near_duplicates(trigrams(data.docs[idx]), trigrams(data.docs[idy]))}")

'-------\\First document\n---------'

"Centrica leaves investors chilly as profit forecast falls short endoftitle Shares in the owner of British Gas have fallen sharply after the company warned of a series of pressures on profits.\n\nCentrica lost 8% of its market value in early trading on Thursday after it revealed the company had lost a further 370,000 customers in its residential supply arm - the country's largest - over the four months to October.\n\nIt blamed its efforts to move households off controversial standard variable tariffs (SVT) ahead of a crackdown by the industry regulator.\n\nThe looming price cap on so-called default tariffs has also placed in jeopardy the planned retail supply merger between fellow 'big six'rivals SSE and npower.\n\nBritish Gas has 3.1 million households on its SVT, down from 4.3 million at the beginning of the year.\n\nIt has also raised prices twice.\n\nThe cap, due to begin on 1 January, would force it to record a £70m financial hit in the first three months of 2019, Centrica said.\n

'-------\nSecond document\n---------'

"Centrica leaves investors chilly as profit forecast falls short endoftitle Centrica lost 8% of its market value in early trading on Thursday after it revealed the company had lost a further 370,000 customers in its residential supply arm - the country's largest - over the four months to October.\n\nIt blamed its efforts to move households off controversial standard variable tariffs (SVT) ahead of a crackdown by the industry regulator.\n\nThe looming price cap on so-called default tariffs has also placed in jeopardy the planned retail supply merger between fellow 'big six'rivals SSE and npower.\n\nBritish Gas has 3.1 million households on its SVT, down from 4.3 million at the beginning of the year.\n\nIt has also raised prices twice.\n\nThe cap, due to begin on 1 January, would force it to record a £70m financial hit in the first three months of 2019, Centrica said.\n\nThe company added that while the sum was in line with previous forecasts, it had brought forward the charge because of

'Are near duplicates: True'

In [51]:
len(trigrams(data.docs[idy]).intersection(trigrams(data.docs[idx]))) / len(trigrams(data.docs[idy]).union(trigrams(data.docs[idx])))


0.789760348583878

# Brute-force approach
The brute force approach is quite simple. It iterates over every possible pair and calculates the similarity function for each of them. This ammounts to $ \frac{n(n-1)}{2}$ comparisons.

In [52]:
dups = set()
for i in fastprogress.progress_bar(range(len(data)-1), total=len(data)-1):
    curr_doc = data.iloc[i].trigrams
    for j in range(i+1, len(data)):
        comparison_doc = data.iloc[j].trigrams
        if are_near_duplicates(curr_doc, comparison_doc):
            dups.add((i,j))

So, in total we have the following number of duplicates:

In [53]:
len(dups)

16625

# Length-based filtering

In length based filtering, we will leverage the fact that sets of different cardinalities have a limit on their Jaccard similarity.

For example, consider two sets: A with size 3, B with size 7. Even if all elements of set A are elements of set B, their Jaccard similarity would be 3/7.
To use this to our advantage, we will only compare documents that are of similar sizes. I.e. a document of size x, will be compared only with documents of size $[x, \left \lfloor x / J\right\rfloor +1]$

Please check the book for the complete details


In [59]:
data['sorted_trigrams'] = data.trigrams.apply(lambda x: sorted(list(x)))
data['trigrams_len'] = data.trigrams.apply(len)

sorted_data = data.sort_values('sorted_trigrams')

In [60]:
dups = set()
num_comparisons = 0
for i in fastprogress.progress_bar(range(len(sorted_data.index))):
    curr = sorted_data.iloc[i]
    cardinality = curr.trigrams_len
    max_cardinality = cardinality / J
    remaining_data = sorted_data.loc[sorted_data.index[i+1:]]
    potential_dups = remaining_data.loc[remaining_data.trigrams_len <= max_cardinality].index
    for j in potential_dups:
        num_comparisons += 1
        other = sorted_data.loc[j]
        if are_near_duplicates(curr.trigrams, other.trigrams):
            dups.add((i,j))

In [65]:
len(data) * len(data) /2

5326848.0

In [63]:
len(dups), num_comparisons

(16625, 3965249)

# Prefix-based filtering

With prefix-based filtering, we rely on the sorted order of the universal set of trigrams. For each document, we sort the trigrams according to the universal order(we just use default lexicographic in this case) and calculate a prefix length (prefix_len) which is dependent on the length of the document and the required Jaccard similarity. 
For each of the first prefix_len trigrams for the document we check the index for that trigram and compare the document to all documents in the corresponding bucket for matches.
After we're done, we add the document to each of the buckets visited in the previous step.

The prefix length is calculated in such a manner to guarantee that all near-duplicates will be found. Please consult the book for the proof and additional details.

In [66]:
from collections import defaultdict
import math
buckets = defaultdict(list)
dups = set()
num_comparisons = 0
for i in fastprogress.progress_bar(range(len(sorted_data.index))):
    curr = sorted_data.iloc[i]
    prefix_len = math.floor((1 - J) *  curr.trigrams_len) + 1
    potentials = set()
    
    for j in range(prefix_len):
        potentials.update(buckets[curr.sorted_trigrams[j]])
    for p in potentials:
        other = sorted_data.iloc[p]
        num_comparisons += 1
        if are_near_duplicates(curr.trigrams, other.trigrams):
            dups.add((i,p))
    for j in range(prefix_len):
        buckets[curr.sorted_trigrams[j]].append(i)
len(dups), num_comparisons

(16625, 2083264)

# Minhash LSH

Minhash is a technique where we hash every document to a vector of numbers. These vectors are constructed in such a way so that similar documents will map to similar vectors. To compute the similarity between documents we will simply compute the similarity between the vectors(aka hashes).

Of course, this doesn't solve the problem of calculating the similarity between every pair of documents. This is where the Locality Sensitive Hashing(LSH) index comes into play. Essentially, the locality sensitive hashing technique maps each hash to a collection of buckets. Afterwards, each document needs only be compared with the documents that fall in the same buckets.

There are a lot of ways to calculate minhashes and they are task dependent. For the sake of argument, we will evaluate the following ways:

1. 9-char shingles - model each document as the set of consecutive 9-chars. E.g. "Hello world" would be modeled as {"Hello wor","ello worl", "llo world" }
2. unigram shingles - model each document as the set of unigrams/words that form it. E.g. "Hello world" would be modeled as {"hello", "world"}
3. bigram shingles - model each document as the set of bigrams that form it. E.g. "Hello world" would be modeled as {"hello world"}
4. trigram shingles - model each document as the set of trigrams that form it. E.g. "One two three four" would be modeled as {"one two three", "two three four"}
5. filtered trigram shingles - the same as 4., but only select trigrams which start with a *stopword*. 


In [71]:
from datasketch import MinHash, MinHashLSH, LeanMinHash

In [95]:
def char_shingles(doc, n_chars=9):
    final_offset = min(len(doc.text), 2000)
    return set(doc.text[i:i+n_chars] for i in range(final_offset-n_chars))

def ngrams(doc, n, filter = lambda x: True):
    for i in range(len(doc)- n + 1):
        if filter(doc[i]):
            yield " ".join([t.lower_ for t in doc[i:i+n]])

def unigram_shingles(doc):
    return {t.lower_ for t in doc}


def bigram_shingles(doc):
    return set(ngrams(doc, 2))

def trigram_shingles(doc):
    return set(ngrams(doc, 3))

def filtered_trigram_shingles(doc):
    return set(ngrams(doc, 3, filter = lambda x: x.is_stop))


In [96]:
def get_mh(shingles):
    mh = MinHash()
    for s in shingles:
        mh.update(s.encode('utf8'))
    return LeanMinHash(mh)

def measure(shingle_fn):
    dups = set()
    num_comparisons = 0
    mhlsh = MinHashLSH(threshold=0.8)
    # build the index
    for k, doc in fastprogress.progress_bar(sorted_data.docs.iteritems(), total=len(sorted_data)):
        shingles = shingle_fn(doc)
        if len(shingles) == 0:
            continue
        mh = get_mh(shingles)
        mhlsh.insert(k, mh)
    # find the duplicates
    for k, curr in fastprogress.progress_bar(sorted_data.iterrows(), total=len(sorted_data)):
        doc = curr.docs
        shingles = shingle_fn(doc)
        mh = get_mh(shingles)
        potentials = mhlsh.query(mh)
        for p in potentials:
            if p == k:
                continue
            other = sorted_data.loc[p]
            num_comparisons += 1
            if are_near_duplicates(curr.trigrams, other.trigrams):
                dups.add((min(k,p), max(k,p)))
    return len(dups), num_comparisons


In [97]:
measure(char_shingles)

(15177, 35596)

In [98]:
measure(unigram_shingles)

(16392, 37768)

In [99]:
measure(bigram_shingles)

(16048, 33838)

In [100]:
measure(trigram_shingles)

(15959, 32774)

In [101]:
measure(filtered_trigram_shingles)

(16307, 35706)